In [3]:
import numpy;
# import matplotlib.pyplot as plt;

# from tensorflow.keras.initializers import TruncatedNormal;

# from tqdm import tqdm;

import numpy as np
import random
from tqdm import tqdm

In [4]:
# Training data
feature = numpy.array(
    [
        [0.08, 0.72], [0.26, 0.58], [0.45, 0.15], 
        [0.60, 0.30], [0.10, 1.0], [0.35, 0.95], 
        [0.70, 0.65], [0.92, 0.45]
    ]);

label = numpy.array([1, 1, 1, 1, -1, -1, -1, -1]);

In [5]:
class AdalineGA:

    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.population_size = 50
        self.generations = 100
        self.mutation_rate = 0.01
        self.crossover_rate = 0.7

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = {
                'weight_x1': np.random.randn(),
                'weight_x2': np.random.randn(),
                'bias': np.random.randn()
            }
            population.append(individual)
        return population

    def fitness(self, individual):
        predictions = []
        for i in self.feature:
            prediction = (individual['weight_x1'] * i[0]) + (individual['weight_x2'] * i[1]) + individual['bias']
            predictions.append(prediction)
        mse = np.mean((self.label - predictions) ** 2)
        return -mse  # We use negative MSE because GA maximizes fitness

    def select(self, population):
        sorted_population = sorted(population, key=self.fitness, reverse=True)
        return sorted_population[:int(0.2 * self.population_size)]  # Select top 20%

    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            crossover_point = random.randint(0, 2)
            child1 = parent1.copy()
            child2 = parent2.copy()
            if crossover_point == 0:
                child1['weight_x1'], child2['weight_x1'] = child2['weight_x1'], child1['weight_x1']
            elif crossover_point == 1:
                child1['weight_x2'], child2['weight_x2'] = child2['weight_x2'], child1['weight_x2']
            else:
                child1['bias'], child2['bias'] = child2['bias'], child1['bias']
            return child1, child2
        else:
            return parent1, parent2

    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            individual['weight_x1'] += np.random.normal()
        if random.random() < self.mutation_rate:
            individual['weight_x2'] += np.random.normal()
        if random.random() < self.mutation_rate:
            individual['bias'] += np.random.normal()
        return individual

    def train(self):
        population = self.initialize_population()
        for generation in tqdm(range(self.generations)):
            new_population = []
            selected_individuals = self.select(population)
            while len(new_population) < self.population_size:
                parent1 = random.choice(selected_individuals)
                parent2 = random.choice(selected_individuals)
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(child1))
                new_population.append(self.mutate(child2))
            population = new_population

        best_individual = max(population, key=self.fitness)
        self.weight_x1 = best_individual['weight_x1']
        self.weight_x2 = best_individual['weight_x2']
        self.bias = best_individual['bias']
        print(f"Best weights - weight_x1: {self.weight_x1}, weight_x2: {self.weight_x2}, bias: {self.bias}")

In [6]:
model = AdalineGA(feature, label);
model.train();

100%|██████████| 100/100 [00:00<00:00, 813.18it/s]

Best weights - weight_x1: -1.5232310024097628, weight_x2: -2.209105990379644, bias: 1.8445584051450141
